In [ ]:
%pip install transformers
%pip install datasets loralib sentencepiece 
%pip install bitsandbytes accelerate
%pip install langchain

# Loading Alpaca 7B

In [ ]:
%pip install protobuf

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    # load_in_8bit=True,
    device_map='auto',
    local_files_only=True
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# Setting up a Chat with memory

In [ ]:

from DBObject import DBObject


In [ ]:
from langchain.utilities import SQLDatabase

def connect_fun(database_name: str) -> DBObject:
  conn = SQLDatabase.from_uri(f"sqlite:///{database_name}?mode=ro") # open in readonly mode
  return DBObject(db=conn, llm=local_llm)

In [ ]:

def query_fun(question: str, tables: list[str], database_connection: DBObject) -> str:
  response = database_connection.make_query(question, tables)

  return response


In [16]:
# test:

connection = connect_fun("sampleDatabases/database_10.db")

print(
  query_fun("How many users have age over 45?", ["users"], connection)
)




> Entering new ConversationChain chain...
Prompt after formatting:
You are an AI chatbot designed to create SQL queries based on a schema, a series of table name hints, and inputs from the user.

Schema:

CREATE TABLE users (
	id INTEGER, 
	name TEXT, 
	age INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from users table:
id	name	age
1	7221ea44	59
2	8dfb994b	11
3	b5c64f0b	66
*/

History: 


Question: How many users have age over 45? 
Table name hints: users
SQL Query:

> Finished chain.
 SELECT COUNT(*) FROM users WHERE age > 45;
